<a href="https://colab.research.google.com/github/keiiigo/RecursosOpenSource/blob/main/Time_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
GRAPH_HOPPER_URL = 'insert your localhost'

CSV_FILE = 'lkmsk.csv'
df = pd.read_csv(CSV_FILE)


def get_duration(lat1, lon1, lat2, lon2, cache):
    key = (lat1, lon1, lat2, lon2)
    if key in cache:
        return cache[key]

    params = {
        'point': [f'{lat1},{lon1}', f'{lat2},{lon2}'],
        'profile': 'car',
        'locale': 'en',
        'key': ''
    }

    try:
        response = requests.get(GRAPH_HOPPER_URL, params=params)

        if response.status_code == 200:
            data = response.json()
            duration_ms = data['paths'][0]['time']
            duration_seconds = duration_ms / 1000
            cache[key] = duration_seconds
            return duration_seconds
        else:
            print(f"Error: {response.status_code}, Response: {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

if 'Latitude' not in df.columns or 'Longitude' not in df.columns:
    raise ValueError("CSV must contain 'Latitude' and 'Longitude' columns.")

locations = [(row['Latitude'], row['Longitude']) for _, row in df.iterrows()]
cache = {}

time_matrix = pd.DataFrame(index=df['Customer Name'], columns=df['Customer Name'], dtype=float)

def compute_duration(i, j, lat1, lon1, lat2, lon2):
    if i == j:
        return 0
    print(f"Calculating duration from Location{i+1} to Location{j+1}...")
    duration = get_duration(lat1, lon1, lat2, lon2, cache)
    if duration is not None:
        print(f"Duration from Location{i+1} to Location{j+1}: {duration} seconds")
    else:
        print(f"Failed to get duration from Location{i+1} to Location{j+1}.")
    return duration

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {}
    for i, (lat1, lon1) in enumerate(locations):
        for j, (lat2, lon2) in enumerate(locations):
            futures[(i, j)] = executor.submit(compute_duration, i, j, lat1, lon1, lat2, lon2)

    for (i, j), future in futures.items():
        time_matrix.iloc[i, j] = future.result()
        sleep(0.05)

OUTPUT_FILE = 'time_matrix.csv'
time_matrix.to_csv(OUTPUT_FILE)
print(f"Time matrix saved to {OUTPUT_FILE}")

print("Time Matrix:")
print(time_matrix)